# Tutorial: Molkit

## Introduction
This tutorial will be focus in tools available in the new ModelHamiltonian module to manipulate molecular hamiltonians.

### 1. Instantiating a Molecular Hamiltonian

In [1]:
import numpy as np
from moha import HamHeisenberg
from moha.molkit.hamiltonians import MolHam

# Define a spin-based (Heisenberg) Hamiltonian as an example.
# Note: You can substitute this with any other supported Hamiltonian type as needed.
J_eq = np.array([[0, 1],
                 [1, 0]])
J_ax = np.array([[0, 0.5],
                 [0.5, 0]])
mu = np.zeros(2)
ham_hei  = HamHeisenberg(J_eq=J_eq, J_ax=J_ax, mu=mu)
one_body = ham_hei.generate_one_body_integral(dense=True)
two_body = ham_hei.generate_two_body_integral(dense=True)

# Instantiate the molecular Hamiltonian using the computed one-body and two-body integrals
Molecular_Hamiltonian = MolHam(one_body=one_body, two_body=two_body)
print(Molecular_Hamiltonian)

### 2. Converting the Hamiltonian to the Spin-Orbital Basis

You can easily perform this conversion by using the `spinize_H` method, which returns the one-body and two-body integrals in the spin-orbital basis.

#### Conversion Rules

- *One-body term:*
  - The spatial one-body integral is mapped to the spin-orbital basis as:
    - $h_{pq}^{\alpha\alpha} = h_{pq}^{\beta\beta} = h_{pq}^{\text{spatial}}$
    - $h_{pq}^{\alpha\beta} = h_{pq}^{\beta\alpha} = 0$

- *Two-body term:*
  - The spatial two-body integral is mapped as:
    - $V_{pqrs}^{\alpha\alpha\alpha\alpha} = V_{pqrs}^{\alpha\beta\alpha\beta} = V_{pqrs}^{\beta\alpha\beta\alpha} = V_{pqrs}^{\text{spatial}}$

In [2]:
one_body_spin, two_body_spin = Molecular_Hamiltonian.spinize_H()
print("One-body integrals (spin basis):\n", one_body_spin)
print("Two-body integrals (spin basis):\n", two_body_spin)

One-body integrals (spin basis):
 [[-0.125  0.     0.     0.     0.     0.     0.     0.   ]
 [ 0.    -0.125  0.     0.     0.     0.     0.     0.   ]
 [ 0.     0.    -0.125  0.     0.     0.     0.     0.   ]
 [ 0.     0.     0.    -0.125  0.     0.     0.     0.   ]
 [ 0.     0.     0.     0.    -0.125  0.     0.     0.   ]
 [ 0.     0.     0.     0.     0.    -0.125  0.     0.   ]
 [ 0.     0.     0.     0.     0.     0.    -0.125  0.   ]
 [ 0.     0.     0.     0.     0.     0.     0.    -0.125]]
Two-body integrals (spin basis):
 [[[[0.    0.    0.    ... 0.    0.    0.   ]
   [0.    0.    0.    ... 0.    0.    0.   ]
   [0.    0.    0.    ... 0.    0.    0.   ]
   ...
   [0.    0.    0.    ... 0.    0.    0.   ]
   [0.    0.    0.    ... 0.    0.    0.   ]
   [0.    0.    0.    ... 0.    0.    0.   ]]

  [[0.    0.125 0.    ... 0.    0.    0.   ]
   [0.    0.    0.    ... 0.    0.    0.   ]
   [0.    0.    0.    ... 0.    0.    0.   ]
   ...
   [0.    0.    0.    ... 0.    0.    

### 3. Antisymmetrizing the Two-Electron Integrals

You can apply proper antisymmetrization to the two-electron integrals using the `antisymmetrize` method. 

This ensures the integrals obey the required permutation symmetry for fermionsm following the rule:
$V_{pqrs} = -V_{pqsr} = -V_{qprs} = V_{qpsr}$

In [3]:
Molecular_Hamiltonian.antisymmetrize()

### 4. Converting the Two-Body Term to the Geminal Basis

The `to_geminal` method converts the two-body term from the spin-orbital basis to the geminal basis.  
You can specify the type of two-body term: `'rdm2'` (two-body reduced density matrix) or `'h2'` (two-body Hamiltonian).

The Hamiltonian in the geminal basis is obtained by the following formula:
$V_{A B} =\frac{1}{2}\left(V_{p q r s}-V_{q p r s}-V_{p q r s}+V_{qprs}\right)$


In [4]:
geminal = Molecular_Hamiltonian.to_geminal(two_body_spin, type='h2')

### 5. Constructing the Reduced 4-Index Hamiltonian Tensor

The `to_reduced` method returns the reduced 4-index Hamiltonian tensor $k_{pqrs}$ in the spin-orbital basis, given the number of electrons $N$.

#### Formula

$$
k_{pqrs} = \frac{1}{2(N-1)} \left( h_{pq} \delta_{rs} + h_{rs} \delta_{pq} \right) + \frac{1}{2} V_{pqrs}
$$

where:
- $h_{pq}$: one-electron integrals (spin-orbital basis)
- $V_{pqrs}$: two-electron integrals (spin-orbital basis)
- $\delta_{pq}$: Kronecker delta

In [5]:
reduced = Molecular_Hamiltonian.to_reduced(n_elec=2)
print(reduced.shape)

(8, 8, 8, 8)


### 6. Extracting Spin Blocks from the Two-Body Spin-Orbital Tensor

After converting the Hamiltonian to the spin-orbital basis, you can extract the main spin blocks from the two-body tensor using the `get_spin_blocks` method, by accesing the dictionaries keys: 'aaaa', 'bbbb', 'abab'

In [6]:
spin_blocks = Molecular_Hamiltonian.get_spin_blocks()
print(spin_blocks.keys())

dict_keys(['aaaa', 'bbbb', 'abab'])


### 7. Computing Energy of a system

In this section, we will compute the ground-state energy of a toy molecular system using the Full Configuration Interaction (FCI) method using the `pyscf` library.

1. Perform an FCI calculation to obtain the wavefunction and reduced density matrices (RDMs).

1. Compute the energy using both spin-orbital and geminal representations to verify consistency.

In [7]:
import numpy as np
from pyscf import fci
from moha.molkit.hamiltonians import MolHam

# One-electron integrals (spatial orbital basis)
h_sp = np.array([[1.0, 0.2],
                 [0.2, 1.5]])

# Two-electron integrals (antisymmetrized, chemist's notation)
V_sp = np.zeros((2, 2, 2, 2))
V_sp[0, 0, 0, 0] = 0.7
V_sp[1, 1, 1, 1] = 0.6
N_ELEC, n_spin  = 2, 2
# Create the molecular Hamiltonian object
# Note: The two-electron integrals should be antisymmetrized.                  
toy = MolHam(h_sp, V_sp)

# Convert spatial integrals to spin-orbital form
h_spin, V_spin = toy.spinize_H()           
norb = h_spin.shape[0]

# Perform Full CI calculation
e_fci, ci_vec = fci.direct_spin1.kernel(h_spin, V_spin, norb, N_ELEC, ecore=0.0)
rdm1s, rdm2s = fci.direct_spin1.make_rdm12s(ci_vec, norb, N_ELEC)

# Build spin-orbital 1-RDM
rdm1 = np.block([
    [rdm1s[0], np.zeros_like(rdm1s[0])],
    [np.zeros_like(rdm1s[1]), rdm1s[1]]
])

# Build spin-orbital 2-RDM
n = norb // 2  # spatial orbitals
rdm2 = np.zeros((2*n, 2*n, 2*n, 2*n))
for i in range(n):
    for j in range(n):
        for k in range(n):
            for l in range(n):
                # αααα
                rdm2[i, j, k, l] = rdm2s[0][i, k, j, l]
                # ββββ
                rdm2[i+n, j+n, k+n, l+n] = rdm2s[2][i, k, j, l]
                # αβαβ
                rdm2[i, j+n, k, l+n] = rdm2s[1][i, k, j, l]
                # βαβα (hermitian transpose of αβαβ)
                rdm2[i+n, j, k+n, l] = rdm2s[1][i, k, j, l]

# Convert to reduced form
H = toy.to_reduced(n_elec=N_ELEC)  # H = h↑ + 0.5 * V
# Energy from spin-orbital formulation
E_spin = np.einsum('pqrs,pqrs', H, rdm2)

# Convert to geminal form
H_gem = toy.to_geminal(H, type='h2')
rdm_gem = toy.to_geminal(rdm2, type='rdm2')
# Energy from geminal formulation
E_gem   = np.einsum('pq,pq', H_gem, rdm_gem)

# Print the energies from both formulations
print("E_spin-orbital :", E_spin)
print("E_geminal      :", E_gem)
assert np.allclose(E_spin, E_gem, atol=1e-10), "Energies do not match!"

E_spin-orbital : 1.004059262354666e-12
E_geminal      : 1.0040474662350293e-12
